Hate Speech Filtering

Using Tf-Idf and BoW to build SVM model

Linear model? Or RBF model?

RACIST texts classified with '1'
NON-RACIST texts classified with '0'


In [61]:
# Libraries


import nltk
nltk.download('wordnet')
import re 

import pandas as pd
import numpy as np

import nltk
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.svm import SVC
# import matplotlib.pyplot as plt

# NLTK Stemmer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Logistic Regression
lreg = LogisticRegression()

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [47]:
# Read data pandas, keep raw copy of data
train_RAW = pd.read_csv('train.csv')
test_RAW = pd.read_csv('test.csv')
train = train_RAW
test = test_RAW

TF-IDF (term frequency-inverse document frequency) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents.
This is done by multiplying two metrics: how many times a word appears in a document, and the inverse document frequency of the word across a set of documents.

Ref: 
    https://monkeylearn.com/blog/what-is-tf-idf/#:~:text=TF%2DIDF%20(term%20frequency%2D,across%20a%20set%20of%20documents


Doc:
    https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [48]:
# TF-IDF freq
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.90, 
    min_df=2, 
    max_features=6000, 
    stop_words='english'
    )

Bag of words is a Natural Language Processing technique of text modelling. 
In technical terms, we can say that it is a method of feature extraction with text data.

Ref: 
    https://www.mygreatlearning.com/blog/bag-of-words/


Doc:
    https://scikit-learn.org/stable/modules/feature_extraction.html

In [49]:
# Bag Of Words Model
bow_vectorizer = CountVectorizer(
    max_df=0.90, 
    min_df=2, 
    max_features=6000, 
    stop_words='english'
    )

Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

Ref/Doc:
    https://scikit-learn.org/stable/modules/svm.html#:~:text=Support%20vector%20machines%20(SVMs)%20are,than%20the%20number%20of%20samples

In [51]:
# Combine training data with test data
allTweets = train.append(test, ignore_index=True, sort=False)

# allTweets.head()

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_10421/2057826554.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  allTweets = train.append(test, ignore_index=True, sort=False)


In [52]:
def fnc_clean_tweet(tweet_txt, pattern):
    r = re.findall(pattern, tweet_txt)
    for i in r:
        tweet_txt = re.sub(i, '', tweet_txt)  # Substitute pattern with blank

    return tweet_txt

In [53]:
# Clean text 
# Add new column 'tidy_tweet'

# Clean user handles (@user_name)
allTweets['tidy_tweet'] = np.vectorize(fnc_clean_tweet)(
    allTweets['tweet'], "@[\w]*") 

# Clean non-alphabets
allTweets['tidy_tweet'] = allTweets['tidy_tweet'].str.replace(
    "[^a-zA-Z#]", " ") 

allTweets.head()

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_10421/4278745381.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  allTweets['tidy_tweet'] = allTweets['tidy_tweet'].str.replace(


,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can t use cause th...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation,factsguide society now #motivation


In [54]:
# Tokenization
tokenized_tweet = allTweets['tidy_tweet'].apply(lambda x: x.split())

tokenized_tweet.head()

0    [when, a, father, is, dysfunctional, and, is, ...
1    [thanks, for, #lyft, credit, i, can, t, use, c...
2                              [bihday, your, majesty]
3    [#model, i, love, u, take, with, u, all, the, ...
4              [factsguide, society, now, #motivation]
Name: tidy_tweet, dtype: object

In [20]:
# # Stemming 
# # TODO: need to be worked on more, words not stemmed properly

# tokenized_tweet = tokenized_tweet.apply(
#     lambda x: [stemmer.stem(i) for i in x])  # stemming


# # Combine tokens back together
# for i in range(len(tokenized_tweet)):
#     tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

# # Replace with stemmed 
# allTweets['tidy_tweet'] = tokenized_tweet


# allTweets.head()

In [68]:
# Using lemmatizer instead of stemmer
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('omw-1.4')
nltk.download('wordnet')
tokenized_tweet = tokenized_tweet.apply(
    lambda x: [lemmatizer.lemmatize(i) for i in x])  

# Combine tokens back together
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

# Replace with lemmatized
allTweets['tidy_tweet'] = tokenized_tweet

allTweets.head()


[nltk_data] Downloading package omw-1.4 to /Users/tonycui/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package wordnet to /Users/tonycui/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can t use cause they...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ur
4,5,0.0,factsguide: society now #motivation,factsguide society now #motivation


In [69]:
# Feature extraction - Bag-of-Words [sklearn’s CountVectorizer]
# ------------------------------------------------------------------------------------------------------------------

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(allTweets['tidy_tweet'])

print(bow)

  (0, 1877)	1
  (0, 4704)	1
  (0, 1542)	1
  (0, 2949)	1
  (0, 4579)	1
  (1, 5329)	1
  (1, 3261)	1
  (1, 1215)	1
  (1, 5624)	1
  (1, 844)	1
  (1, 1519)	1
  (1, 3748)	1
  (1, 5644)	1
  (1, 3927)	1
  (2, 513)	1
  (2, 3284)	1
  (3, 3487)	1
  (3, 3221)	1
  (3, 5394)	1
  (3, 5619)	1
  (4, 1822)	1
  (4, 4912)	1
  (4, 3528)	1
  (5, 2598)	1
  (5, 1850)	1
  :	:
  (49156, 4633)	1
  (49156, 3130)	1
  (49156, 5900)	2
  (49156, 1426)	1
  (49156, 2515)	1
  (49156, 5625)	1
  (49156, 989)	1
  (49156, 286)	1
  (49156, 3756)	1
  (49156, 5337)	1
  (49157, 2414)	1
  (49157, 5903)	2
  (49157, 1096)	1
  (49157, 4526)	1
  (49157, 3453)	2
  (49157, 3050)	1
  (49157, 1248)	1
  (49157, 1415)	1
  (49157, 3817)	1
  (49158, 2060)	1
  (49158, 3653)	1
  (49158, 4931)	1
  (49158, 1537)	1
  (49158, 2214)	1
  (49158, 3658)	1


In [70]:

# TF-IDF Features - Looks at frequency of occurence for terms/importance of the term

# TF = (Number of times term t appears in a document)/(Number of terms in the document)
# IDF = log(N/n), where, N is the number of documents and n is the number of documents a term t has appeared in.
# 
# TF-IDF = TF*IDF

# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(allTweets['tidy_tweet'])

print(tfidf)


  (0, 4579)	0.4024983119617675
  (0, 2949)	0.36455809535458955
  (0, 1542)	0.5730362603952134
  (0, 4704)	0.5410298170296455
  (0, 1877)	0.2898425181654669
  (1, 3927)	0.40479498927673024
  (1, 5644)	0.4003549376374816
  (1, 3748)	0.3325307874161632
  (1, 1519)	0.2057834060163094
  (1, 844)	0.29563720941752136
  (1, 5624)	0.28022184986744003
  (1, 1215)	0.3584978041999103
  (1, 3261)	0.4153857597592834
  (1, 5329)	0.23652009646579678
  (2, 3284)	0.8801390220492387
  (2, 513)	0.4747160223398928
  (3, 5619)	0.5647875413808863
  (3, 5394)	0.45621564623313093
  (3, 3221)	0.37161906008001283
  (3, 3487)	0.5786031380992227
  (4, 3528)	0.45998601499572633
  (4, 4912)	0.5851033747216161
  (4, 1822)	0.6678824049918725
  (5, 3920)	0.27416804146586254
  (5, 888)	0.44221772005280136
  :	:
  (49156, 989)	0.28164252510602494
  (49156, 5625)	0.251826461349804
  (49156, 2515)	0.2583320003151624
  (49156, 1426)	0.21414209466080109
  (49156, 5900)	0.43346666874721856
  (49156, 3130)	0.15622420919684082


In [71]:
train_bow = bow[:31962, :]
test_bow = bow[31962:, :]

# splitting data into training and validation set



# xtrain_bow training dataset
# xvalid_bow validation for training
# ytrain label vector
# yvalid validation label vector
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(
    train_bow, train['label'], test_size=0.1)

# train_bow feature matrix
# train['label'] label vector
# random_state shuffles data before split into training and testing
# test_size percentage of data gets tested on (0.9 training)


# print(xtrain_bow)
# print(xvalid_bow)
print(ytrain)
# print(yvalid)

# lreg = LogisticRegression()
# lreg.fit(xtrain_bow, ytrain)    
# train using training dataset and label vector

18711    0
4487     0
9172     0
11563    0
8731     0
        ..
7751     0
17700    0
1248     0
22490    0
16050    0
Name: label, Length: 28765, dtype: int64


In [73]:
# tfidf feature matrix
train_tfidf = tfidf[:31962, :]
test_tfidf = tfidf[31962:, :]


# ytrain.index index of axis labels 
xtrain_tfidf = train_tfidf[ytrain.index]
xvalid_tfidf = train_tfidf[yvalid.index]

# extract label vectors from the feature matrix via matching index

# print(xtrain_tfidf)


# fit on logistic regression
# lreg.fit(xtrain_tfidf, ytrain) 

In [85]:
# SVM
svm = SVC(kernel='rbf')

SVM tutorial: https://www.youtube.com/watch?v=FB5EdxAGxQg

In [86]:
# TODO: understand how svm model is built using BoW and TF-IDF

svm.fit(xtrain_bow, ytrain)        # Build using bow
# accuracy test
svm.score(xvalid_bow, yvalid)       

0.9562089458867689

In [87]:
svm.fit(xtrain_tfidf, ytrain)      # Build using TD-IDF
svm.score(xvalid_tfidf, yvalid)


0.960275258054426